# Trying to build a better function to handle RPM

Shoudl take a dict of things...


In [16]:
from bacpypes.pdu import Address
from bacpypes.object import get_object_class, get_datatype
from bacpypes.apdu import (
    PropertyReference,
    ReadAccessSpecification,
    ReadPropertyRequest,
    ReadPropertyMultipleRequest,
    RejectReason,
    AbortReason,
    RejectPDU,
    AbortPDU,
)
from bacpypes.basetypes import PropertyIdentifier


In [17]:

    _rpm = {'11:2': [("analogInput",1),("analogInput",2),("analogInput",1, "units", "description")]
    }

    _rpm = {'address': '11:2',
            'objects': {'analogInput:1': ['presentValue', 'description', 'unit', 'objectList@idx:0'],
                        'analogInput:2': ['presentValue', 'description', 'unit', 'objectList@idx:0'],
            },
            'vendor_id': 842,
    }


In [18]:
_rpm

{'address': '11:2',
 'objects': {'analogInput:1': ['presentValue',
   'description',
   'unit',
   'objectList@idx:0'],
  'analogInput:2': ['presentValue',
   'description',
   'unit',
   'objectList@idx:0']},
 'vendor_id': 842}

In [35]:

def build_rpm_request_from_dict(request_dict):
    """
    Read property multiple allow to read a lot of properties with only one request
    The existing RPM function is made using a string that must be created using bacpypes 
    console style and is hard to automate.

    This new version will be an attempt to improve that::

        _rpm = {'address': '11:2',
            'objects': {'analogInput:1': ['presentValue', 'description', 'unit', 'objectList@idx:0'],
                        'analogInput:2': ['presentValue', 'description', 'unit', 'objectList@idx:0'],
            },
            vendor_id: 842
            }

    """
    vendor_id = 842
    addr = request_dict['address']
    objects = request_dict['objects']
    if 'vendor_id' in request_dict.keys():
        vendor_id = int(request_dict['vendor_id'])

    read_access_spec_list = []
    
    for obj, list_of_properties in objects.items():
        obj_type, obj_instance = obj.split(':')
        obj_type = validate_object_type(obj_type, vendor_id=vendor_id)
        obj_instance = int(obj_instance)
        property_reference_list = build_property_reference_list(obj_type, list_of_properties)
        read_acces_spec = build_read_access_spec(obj_type, obj_instance, property_reference_list)
        read_access_spec_list.append(read_acces_spec)

    if not read_access_spec_list:
        raise RuntimeError("at least one read access specification required")

    # build the request
    request = ReadPropertyMultipleRequest(
        listOfReadAccessSpecs=read_access_spec_list
    )
    request.pduDestination = Address(addr)

    return request


def validate_object_type(obj_type, vendor_id=842):
    if obj_type.isdigit():
        obj_type = int(obj_type)
    elif "@obj_" in obj_type:
        obj_type = int(obj_type.split("_")[1])
    elif not get_object_class(obj_type, vendor_id=vendor_id):
        raise ValueError("Unknown object type : {}".format(obj_type))
    return obj_type


def build_read_access_spec(obj_type, obj_instance, property_reference_list):
    return ReadAccessSpecification(
                objectIdentifier=(obj_type, obj_instance),
                listOfPropertyReferences=property_reference_list,
            )

def build_property_reference_list(obj_type, list_of_properties):
    property_reference_list = []
    for prop in list_of_properties:
        idx = None
        if "@idx:" in prop:
            prop, idx = prop.split('@idx:')
        prop_id = validate_property_id(obj_type, prop)
        prop_reference = PropertyReference(propertyIdentifier=prop_id)
        if idx:
            prop_reference.propertyArrayIndex = int(idx)
        property_reference_list.append(prop_reference)
    return property_reference_list

def validate_property_id(obj_type, prop_id):
    if prop_id in PropertyIdentifier.enumerations:
        if prop_id in (
                    "all",
                    "required",
                    "optional",
                    "objectName",
                    "objectType",
                    "objectIdentifier",
                    "polarity",
                ): 
                return prop_id
        elif validate_datatype(obj_type, prop_id):       
            return prop_id  
        else:
            raise ValueError("invalid property for object type : {} | {}".format(obj_type, prop_id))
    elif "@prop_" in prop_id:
        return int(prop_id.split("_")[1])
    #elif "@obj_" in prop_id:
    else:
        raise ValueError('{} is an invalid property for {}'.format(prop_id, obj_type))


def validate_datatype(obj_type, prop_id, vendor_id=842):
    return get_datatype(obj_type, prop_id, vendor_id=vendor_id) if not None else False


In [36]:
_rpm = {'address': '11:2',
            'objects': {'analogInput:1': ['presentValue', 'description', 'units', 'eventMessageTexts@idx:0'],
                        'analogInput:2': ['presentValue', 'description', 'units', 'eventMessageTexts@idx:0'],
            }
}

rpm = build_rpm_request(_rpm)

In [37]:
rpm.debug_contents()

    pduDestination = <Address 11:2>
    pduExpectingReply = 1
    pduNetworkPriority = 0
    apduType = 0
    apduService = 14
    listOfReadAccessSpecs
        [0]            objectIdentifier = ('analogInput', 1)
            listOfPropertyReferences
                [0]                    propertyIdentifier = 'presentValue'
                [1]                    propertyIdentifier = 'description'
                [2]                    propertyIdentifier = 'units'
                [3]                    propertyIdentifier = 'eventMessageTexts'
                    propertyArrayIndex = 0
        [1]            objectIdentifier = ('analogInput', 2)
            listOfPropertyReferences
                [0]                    propertyIdentifier = 'presentValue'
                [1]                    propertyIdentifier = 'description'
                [2]                    propertyIdentifier = 'units'
                [3]                    propertyIdentifier = 'eventMessageTexts'
              

In [1]:
import BAC0
bacnet = BAC0.lite()



2020-11-08 20:37:03,777 - INFO    | Starting BAC0 version 20.11.04dev (Lite)
2020-11-08 20:37:03,778 - INFO    | Use BAC0.log_level to adjust verbosity of the app.
2020-11-08 20:37:03,779 - INFO    | Ex. BAC0.log_level('silence') or BAC0.log_level('error')
2020-11-08 20:37:03,780 - INFO    | Starting TaskManager
2020-11-08 20:37:03,906 - INFO    | Using ip : 192.168.211.54
2020-11-08 20:37:03,908 - INFO    | Starting app...
2020-11-08 20:37:03,910 - INFO    | BAC0 started
2020-11-08 20:37:03,911 - INFO    | Registered as Simple BACnet/IP App
2020-11-08 20:37:03,943 - INFO    | Update Local COV Task started


{('analogInput', 1094): [('presentValue', 4.233697891235352),
  ('statusFlags', [0, 0, 0, 0]),
  ('units', 'pascals'),
  ('description', 'Discharge Air Velocity Pressure')],
 ('analogValue', 4410): [('presentValue', 0.005016503389924765),
  ('statusFlags', [0, 0, 1, 0]),
  ('units', 'percent'),
  ('description', '')]}

In [2]:
_rpm = {'address': '303:9', 'objects': {'analogInput:1094': ['objectName', 'presentValue', 'statusFlags', 'units','description'], 'analogValue:4410': ['objectName', 'presentValue', 'statusFlags', 'units', 'description']}}

bacnet.readMultiple('303:9', request_dict=_rpm)

{('analogInput', 1094): [('objectName', 'DA-VP'),
  ('presentValue', 4.1766886711120605),
  ('statusFlags', [0, 0, 0, 0]),
  ('units', 'pascals'),
  ('description', 'Discharge Air Velocity Pressure')],
 ('analogValue', 4410): [('objectName', 'SAFLOW-ABSEFFORT'),
  ('presentValue', 0.004353651311248541),
  ('statusFlags', [0, 0, 1, 0]),
  ('units', 'percent'),
  ('description', '')]}

In [40]:
build_rpm_request_from_dict(_rpm)

ValueError:  description is an invalid property for analogInput

In [1]:
range(0,129)

range(0, 129)

In [2]:
req = {}
req['address'] = '11:2'
req['objects'] = {}

for each in range(0,129):
    req['objects']['analogInput:{}'.format(each)] = ['presentValue']
req

{'address': '11:2',
 'objects': {'analogInput:0': ['presentValue'],
  'analogInput:1': ['presentValue'],
  'analogInput:2': ['presentValue'],
  'analogInput:3': ['presentValue'],
  'analogInput:4': ['presentValue'],
  'analogInput:5': ['presentValue'],
  'analogInput:6': ['presentValue'],
  'analogInput:7': ['presentValue'],
  'analogInput:8': ['presentValue'],
  'analogInput:9': ['presentValue'],
  'analogInput:10': ['presentValue'],
  'analogInput:11': ['presentValue'],
  'analogInput:12': ['presentValue'],
  'analogInput:13': ['presentValue'],
  'analogInput:14': ['presentValue'],
  'analogInput:15': ['presentValue'],
  'analogInput:16': ['presentValue'],
  'analogInput:17': ['presentValue'],
  'analogInput:18': ['presentValue'],
  'analogInput:19': ['presentValue'],
  'analogInput:20': ['presentValue'],
  'analogInput:21': ['presentValue'],
  'analogInput:22': ['presentValue'],
  'analogInput:23': ['presentValue'],
  'analogInput:24': ['presentValue'],
  'analogInput:25': ['present